In [1]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras import optimizers
from keras.models import Model

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '../data/train'
validation_data_dir = '../data/validation'

In [3]:
model_vgg = applications.VGG16(include_top=False, weights='imagenet', input_shape=(150, 150, 3))

In [4]:
datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size)

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size)

Found 40805 images belonging to 25 classes.
Found 8856 images belonging to 25 classes.


In [5]:
epochs = 30
train_samples = 40805
validation_samples = 8856

In [6]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model_vgg.output_shape[1:]))
top_model.add(Dense(512, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(25, activation='softmax'))

top_model.load_weights('../models/bottleneck_30_epochs.h5')

model = Model(inputs=model_vgg.input, outputs=top_model(model_vgg.output))

In [7]:
for layer in model.layers[:15]:
    layer.trainable = False

In [8]:
# initiate RMSprop optimizer
opt = optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [ ]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy'])

In [ ]:
# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_samples // batch_size)

Epoch 1/30
 187/1275 [===>..........................] - ETA: 569s - loss: 1.4441 - acc: 0.7468 - top_k_categorical_accuracy: 0.9106

In [ ]:
model.save_weights('../models/finetuning_30_epochs.h5')

In [ ]:
# model.load_weights('../models/finetuning_30_epochs.h5)

In [ ]:
model.evaluate_generator(validation_generator, validation_samples / batch_size, workers=12)